In [3]:
import os
import cv2
import cvlib as cv
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model
import numpy as np
from matplotlib import pyplot as plt
from mtcnn import MTCNN
from keras.utils import np_utils
from sklearn.metrics import f1_score as sklearn_f1_score
import cv2
import numpy as np
from keras.models import load_model
from mtcnn import MTCNN
import pyttsx3


In [6]:
def crop_images(input_dir, out_dir):
    source_list = os.listdir(input_dir)
    for f in source_list:
        f_name = os.path.join(input_dir, f)
        image = cv2.imread(f_name)

        # apply face detection
        faces, confidences = cv.detect_face(image)

        print(faces)
        print(confidences)

        # loop through detected faces
        for face, conf in zip(faces, confidences):
            (startX, startY) = face[0], face[1]
            (endX, endY) = face[2], face[3]
            # draw rectangle over face
            rectangle = cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

            # Check if the bounding box is within the image's dimensions
            if 0 <= startX < image.shape[1] and 0 <= endX < image.shape[1] and 0 <= startY < image.shape[0] and 0 <= endY < image.shape[0]:
                cropped_face = image[startY:endY, startX:endX]
                # save output
                cv2.imwrite(os.path.join(out_dir, f), cropped_face)
            else:
                print(f"The bounding box of face in {f} is out of the image's boundaries.")

crop_images('C:/Users/mupaj/Documents/Apps/final_vc/FaceData/woman',
            "C:/Users/mupaj/Documents/Apps/final_vc/FaceData/woman_croped/")

crop_images('C:/Users/mupaj/Documents/Apps/final_vc/FaceData/man',
            "C:/Users/mupaj/Documents/Apps/final_vc/FaceData/man_croped/")


[[197, 116, 413, 426]]
[0.9999559]
[[59, 41, 177, 197], [136, 123, 251, 272]]
[0.9999906, 0.9960616]
The bounding box of face in download (1).jpg is out of the image's boundaries.
[[69, 72, 147, 166]]
[0.99962497]
[[74, 5, 132, 83]]
[0.99676067]
[[93, 46, 171, 143]]
[0.9999738]
[[87, 32, 163, 140]]
[0.99924755]
[[52, 42, 168, 199], [128, 120, 242, 274]]
[0.9999573, 0.9978136]
The bounding box of face in download (6).jpg is out of the image's boundaries.
[[41, 39, 139, 184], [99, 130, 200, 273]]
[0.999928, 0.8085954]
The bounding box of face in download.jpg is out of the image's boundaries.
[[73, 28, 142, 130]]
[0.99999106]
[[192, 27, 268, 127], [27, 31, 106, 134]]
[0.99991834, 0.99930406]
[[59, 55, 126, 148]]
[0.9999901]
[[73, 36, 153, 150]]
[0.99967897]
[[34, 21, 151, 171], [113, 91, 229, 241]]
[0.9875519, 0.92625755]
The bounding box of face in images (13).jpg is out of the image's boundaries.
[[75, 30, 174, 177], [159, 90, 259, 242]]
[0.99963117, 0.93518806]
The bounding box of face

In [7]:
"""
Resizing Cropped Images and save them
"""
def resize_image(input_dir,output_dir):

    # specify the new dimensions you want for the images
    new_dimensions = (40, 40)  # you should change this to the dimensions you want

    # create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # add/modify image file types here
            img = Image.open(os.path.join(input_dir, filename))
            img.thumbnail(new_dimensions)
            img.save(os.path.join(output_dir, filename))

resize_image("C:/Users/mupaj/Documents/Apps/final_vc/FaceData/man_croped/","C:/Users/mupaj/Documents/Apps/final_vc/FaceData/man_resized/")

resize_image("C:/Users/mupaj/Documents/Apps/final_vc/FaceData/woman_croped/","C:/Users/mupaj/Documents/Apps/final_vc/FaceData/woman_resized/")

In [8]:
"""
Find F1 Score
"""

def compute_f1_score(test_generator, model):

    # Generate a list of labels
    test_labels = test_generator.classes
    # Convert the labels to categorical
    test_labels = np_utils.to_categorical(test_labels, num_classes=len(test_generator.class_indices))

    # Make the model predictions
    predictions = model.predict(test_generator)

    # Convert predictions classes to one hot vectors
    predicted_classes = np.argmax(predictions, axis=1)
    # Convert test observations to one hot vectors
    true_classes = np.argmax(test_labels, axis=1)

    # compute the confusion matrix
    f1Score = sklearn_f1_score(true_classes, predicted_classes, average='macro')

    print("F1 Score: ", f1Score)



In [9]:
"""
Train Model
"""
def train_model(train_dir,validation_dir,test_dir):

    # Define ImageDataGenerators for training, validation and testing
    # This will handle preprocessing, image augmentation, and splitting.
    datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

    # Create the model architecture.
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(len(train_generator.class_indices), activation='softmax'))

    # Compile the model.
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model.
    model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)

    # Save the model for later use.
    model.save('model.h5')

    compute_f1_score(test_generator,model)

    return train_generator, validation_generator, test_generator, model



train_generator, validation_generator, test_generator, model = train_model("C:/Users/mupaj/Documents/Apps/final_vc/FaceData/train","C:/Users/mupaj/Documents/Apps/final_vc/FaceData/val","C:/Users/mupaj/Documents/Apps/final_vc/FaceData/test")

Found 57 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 2s 625ms/step - loss: 2.7030 - accuracy: 0.5263 - val_loss: 3.6329 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 1s 500ms/step - loss: 2.1968 - accuracy: 0.4912 - val_loss: 0.7737 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 1s 543ms/step - loss: 0.6319 - accuracy: 0.5088 - val_loss: 0.8595 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 1s 463ms/step - loss: 0.5553 - accuracy: 0.6316 - val_loss: 0.6348 - val_accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 1s 433ms/step - loss: 0.4775 - accuracy: 0.7895 - val_loss: 0.5559 - val_accuracy: 0.6667
Epoch 6/10
2/2 [==============================] - 1s 503ms/step - loss: 0.3313 - accuracy: 0.9649 - val_loss: 0.6845 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==========================

In [10]:
"""
Load Model and predict face
"""
# Load the trained model
model = load_model('model.h5')

# Initialize the MTCNN detector for face detection
detector = MTCNN()


def predict_face(image_path, train_generator):
    # Load the image
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    # Use MTCNN to detect faces in the image
    result = detector.detect_faces(image)
    # If a face is detected
    if result:
        # Get the bounding box coordinates
        x, y, width, height = result[0]['box']
        # Crop the face from the image
        face = image[y:y + height, x:x + width]
        # Resize the face to the size your model expects
        face = cv2.resize(face, (150, 150))
        # Normalize the face
        face = face / 255.0
        # Expand dimensions to match the shape the model expects
        face = np.expand_dims(face, axis=0)

        # Use the model to predict the face
        prediction = model.predict(face)
        label = np.argmax(prediction)

        # Get the label name from the train generator's class_indices
        label_name = list(train_generator.class_indices.keys())[
            list(train_generator.class_indices.values()).index(label)]

        # Draw a rectangle around the face and add a label
        cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(image, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Convert the image color back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Save the image
        cv2.imwrite('C:/Users/mupaj/Documents/Visual Computing/Visual_Computing/data/multi_class_face/val/jolie/1_edited.jpeg',
                    image)

        print(label_name)

        # Show the image with the face detected
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()




In [11]:
predict_face("C:/Users/mupaj/Documents/Visual Computing/Visual_Computing/data/face/storm.jpg", train_generator)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [12]:
"""
Face detection with video
"""

def predict_face_video(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    while(video.isOpened()):
        # Read a frame
        ret, frame = video.read()
        if not ret:
            break

        # Convert the frame color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MTCNN to detect faces in the frame
        result = detector.detect_faces(frame_rgb)

        for i in range(len(result)):
            # Get the bounding box coordinates
            x, y, width, height = result[i]['box']
            # Crop the face from the frame
            face = frame_rgb[y:y + height, x:x + width]
            # Resize the face to the size your model expects
            face = cv2.resize(face, (150, 150))
            # Normalize the face
            face = face / 255.0
            # Expand dimensions to match the shape the model expects
            face = np.expand_dims(face, axis=0)

            # Use the model to predict the face
            prediction = model.predict(face)
            label = np.argmax(prediction)

            # Get the label name from the train generator's class_indices
            label_name = list(train_generator.class_indices.keys())[
                list(train_generator.class_indices.values()).index(label)]

            # Draw a rectangle around the face and add a label
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)

        # Break the loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the video capture and video write objects
    video.release()

    # Close all the frames
    cv2.destroyAllWindows()


predict_face_video('C:/Users/mupaj/Documents/Visual Computing/Visual_Computing/data/face_video/20230519_131703.mp4')


In [13]:
"""
Detect Face via webcam
"""

def predict_face_webcam():
    # Open the webcam
    video = cv2.VideoCapture(0)

    while True:
        # Read a frame
        ret, frame = video.read()

        # Convert the frame color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MTCNN to detect faces in the frame
        result = detector.detect_faces(frame_rgb)

        for i in range(len(result)):
            # Get the bounding box coordinates
            x, y, width, height = result[i]['box']
            # Crop the face from the frame
            face = frame_rgb[y:y + height, x:x + width]
            # Resize the face to the size your model expects
            face = cv2.resize(face, (150, 150))
            # Normalize the face
            face = face / 255.0
            # Expand dimensions to match the shape the model expects
            face = np.expand_dims(face, axis=0)

            # Use the model to predict the face
            prediction = model.predict(face)
            label = np.argmax(prediction)

            # Get the label name from the train generator's class_indices
            label_name = list(train_generator.class_indices.keys())[
                list(train_generator.class_indices.values()).index(label)]

            # Draw a rectangle around the face and add a label
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)

        # Break the loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the video capture
    video.release()

    # Close all the frames
    cv2.destroyAllWindows()


In [14]:
predict_face_webcam()


1/1 [==============================] - 0s 18ms/step


C:\Users\mupaj\Documents\Apps\final_vc\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Apps\final_vc\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Apps\final_vc\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Apps\final_vc\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 

In [23]:

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


# Load the list of class names that YOLO can detect
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Load the trained model
model = load_model('model.h5')

# Load MTCNN
detector = MTCNN()

In [38]:
# Initialize the speech engine
engine = pyttsx3.init()

def text_to_voice(face_name,detected_object):
    # Prepare the text
    text = f"{face_name} You are by a {detected_object}"

    # Say the text
    engine.say(text)

    # Wait until the text has been spoken
    engine.runAndWait()

# Define the handler functions
def handle_dog():
    print("Dog detected!")

def handle_car():
    print("Car detected!")

def handle_chair():
    print("handle_chair detected!")

def handle_bottler():
    print("handle_chair detected!")

# Create a dictionary mapping class_ids to functions
class_handlers = {39: handle_bottler, 56: handle_chair, 16: handle_dog, 2:
    handle_car}

In [44]:
def predict_face_webcam():
    # Open the webcam
    video = cv2.VideoCapture(0)

    while True:
        # Read a frame
        ret, frame = video.read()

        # Convert the frame color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MTCNN to detect faces in the frame
        result = detector.detect_faces(frame_rgb)

        # Process each detected face
        for i in range(len(result)):
            # Get the bounding box coordinates
            x, y, width, height = result[i]['box']
            # Crop the face from the frame
            face = frame_rgb[y:y + height, x:x + width]
            # Resize the face to the size your model expects
            face = cv2.resize(face, (150, 150))
            # Normalize the face
            face = face / 255.0
            # Expand dimensions to match the shape the model expects
            face = np.expand_dims(face, axis=0)

            # Use the model to predict the face
            prediction = model.predict(face)
            label = np.argmax(prediction)

            # Get the label name from the train generator's class_indices
            label_name = list(train_generator.class_indices.keys())[
                list(train_generator.class_indices.values()).index(label)]

            # Draw a rectangle around the face and add a label
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Run the frame through YOLO
        height, width, channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Process the detections
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id != 0:  # Exclude 'person' class
                    # Object bounding box coordinates
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

                    # Call the handler function for this class_id, if one exists
                    if class_id in class_handlers:
                        text_to_voice(label_name,classes[class_id])
                        class_handlers[class_id]()

        # Draw bounding box for each detected object
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)


        # Break the loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the video capture
    video.release()

    # Close all the frames
    cv2.destroyAllWindows()

In [45]:
predict_face_webcam()

1/1 [==============================] - 0s 28ms/step
handle_chair detected!
1/1 [==============================] - 0s 39ms/step
handle_chair detected!
handle_chair detected!
1/1 [==============================] - 0s 26ms/step
handle_chair detected!
handle_chair detected!
handle_chair detected!
handle_chair detected!
1/1 [==============================] - 0s 30ms/step
handle_chair detected!
handle_chair detected!
handle_chair detected!
handle_chair detected!
1/1 [==============================] - 0s 58ms/step


KeyboardInterrupt: 